# Strategy Backtesting

This notebook evaluate the performance of our momentum based strategy using historical market data to simulate real-world execution.

In [ ]:
# requirements 

import pandas as pd
import numpy as np
import os
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import Gateway as gw # Gateway class reads and feeds market data into the system incrementally
import MatchingEngine as me # MatchingEngine class simulates order matching and execution outcomes